# <font color=#c51b8a>VPOD 'Mine-n-Match':</font>
## <font color=#c994c7>Part 1 Objective</font> - Use Species Names from Microspectrophotemetry Data Sheet to Query NCBI for All Related Opsin Sequences  

In [1]:
import os
import re
import datetime
import time
import pandas as pd
from deepBreaks.preprocessing import read_data
from Bio import Entrez, SeqIO
email = 'sethfrazer@ucsb.edu'

In [2]:
from mnm_scripts.mine_n_match_functions import ncbi_fetch_opsins, merge_accessory_dbs

## <font color=#c51b8a>Load data-tables with all of the species and sequence data from accessory lmax databases</font> 

### <font color=#c994c7>VPOD Single Cell Microspectrophotmetry (SCP) Datatable </font>
### In this case our dataframe does not have full species name in one column so we must create a list by directly combining the genus and species names. Then filter to create a list of all unique species names 


In [14]:
report_dir = './data_sources/lmax'
species_list = []
df_list = []

In [ ]:
scp_file = f'{report_dir}/vpod/scp_raw.tsv'
scp_df = pd.read_csv(scp_file, index_col=0, sep='\t')
scp_df['Full_Species'] = (scp_df['Genus'] + ' ' + scp_df['Species']).to_list()
df_list.append(scp_df)
scp_df.head()

In [16]:
species_list+=scp_df['Full_Species'].to_list()

### <font color=#c994c7>Longcore - 'Animal Photopigments' Datatable</font>

### In this next case our dataframe has the full species name in one column so we can create a list directly and filter to create a list of all unique species names


In [ ]:
longcore_file = f'{report_dir}/longcore_data/AnimalPhotopigmentsV1_1.csv'
longcore_df = pd.read_csv(longcore_file, index_col=0)
df_list.append(longcore_df)
longcore_df.head()

In [18]:
species_list+=longcore_df['Full_Species'].to_list()

### <font color=#c994c7>Murphy and Westerman Datatable</font>

In [ ]:
murphy_westerman_file = f'{report_dir}/murphy_westerman/Murphy and Westerman.csv'
murphy_westerman_df = pd.read_csv(murphy_westerman_file, index_col=0)
df_list.append(murphy_westerman_df)
murphy_westerman_df.head()

In [9]:
species_list+=murphy_westerman_df['Full_Species'].to_list()

### <font color=#c994c7>Caves 'Fish' Datatable</font>

In [ ]:
caves_fish_db_file = f'{report_dir}/caves/caves_fish_db.csv'
caves_df = pd.read_csv(caves_fish_db_file, index_col=0)
caves_df['Full_Species'] = (caves_df['Genus'] + ' ' + caves_df['Species']).to_list()
df_list.append(caves_df)
caves_df.head()

In [11]:
species_list+=caves_df['Full_Species'].to_list()

### <font color=#c994c7>Morgan Porter's Datatables</font>

In [ ]:
porter_file1 = f'{report_dir}/megan_porter/megan_porter_extract_clean_2005.csv'
porter_df1 = pd.read_csv(porter_file1, index_col=0)
porter_df2 = pd.read_csv(f'{report_dir}/megan_porter/porter_2006_table1_clean.tsv', sep='\t', index_col=0)
df_list.append(porter_df1)
df_list.append(porter_df2)
porter_df1.head()

In [ ]:
porter_df2.head()

In [14]:
species_list += (porter_df1['Full_Species'].to_list() + porter_df2['Full_Species'].to_list())

### <font color=#c994c7>Merge Accessory Databases</font>

In [ ]:
# Call the function to merge all the species, lambdamax, and potential accession information into one dataframe
merged_df = merge_accessory_dbs(df_list, report_dir)
merged_df.head()

In [ ]:
merged_df.shape

### <font color=#c994c7>All unique species names have been extracted from accessory databases. Now we iteratively query NCBI for opsins from each species.</font>

In [ ]:
len(species_list)

In [ ]:
species_list = list(set(species_list))
len(species_list)

In [ ]:
ncbi_query_df = ncbi_fetch_opsins(email, job_label='mnm_on_all_dbs', out='mnm_on_all_dbs', species_list=species_list)

In [ ]:
ncbi_query_df.head()

In [ ]:
ncbi_query_df.shape

## <font color=#c994c7>Part 2 Objective</font> - Match sequence to it's closest MSP value based on OPTICS predictions

We'll need to...

- Query OPTICS with all the sequence data [bootstrap enabled]
- Extract Predictions
- Match to closest MSP value species-by-species [will need a list of the unique species names] // Match to MaxId as the foreign key

In [1]:
import os
import time 
import datetime
import warnings
import pandas as pd
from deepBreaks.preprocessing import read_data
from mnm_scripts.mine_n_match_functions_old import mine_n_match, post_process_matching, get_prots_from_acc

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [2]:
report_dir = 'mnm_data/mnm_on_mnm_on_all_dbs_2024-12-10_23-31-03' #re-define the report directory if needed
ncbi_q_file = f'./{report_dir}/mnm_on_all_dbs_ncbi_q_data_cleaned.csv'
ncbi = pd.read_csv(ncbi_q_file)
ncbi.head()

,Accession,Genus,Species,Full_Species,Protein,Gene_Description
0,MN519158.1,Carcharhinus,melanopterus,Carcharhinus melanopterus,MNGTEGENFYVPFSNKTGVVRSPFEYPQYYLAEPWQFSIIAAYVFL...,Carcharhinus melanopterus isolate U16228 green...
1,MN519147.1,Carcharhinus,melanopterus,Carcharhinus melanopterus,MNGTEGENFYVPMSNKTGVVRSPFEYSQHYLAEPWMFSVLTAYMFF...,Carcharhinus melanopterus isolate CL4386 rhodo...
2,XM_022258443.2,Pieris,rapae,Pieris rapae,MMQITKIILILIPIIVPCDNSVTNELDIKCVKKSVLTKVYCTNLVR...,"PREDICTED: Pieris rapae opsin, ultraviolet-sen..."
3,AB208675.1,Pieris,rapae,Pieris rapae,MFDTVNATADGGAIAYAFKMVSSEVQENMLGFNIPPEHQDLVHEHW...,"Pieris rapae PrB mRNA for opsin, complete cds"
4,AB208674.1,Pieris,rapae,Pieris rapae,MELNYTAGDPIAFPFKMVSGEVQQHMLGWNIPAEHQGLVHEHWRQF...,"Pieris rapae PrV mRNA for opsin, complete cds"


In [3]:
ncbi.shape

(3359, 6)

In [3]:
# Example use for OPTICS
# python optics_predictions.py -in msp_mined_seqs.fasta -rd mined_msp_seqs -out mined_seq_predictions.tsv -m wildtype -e aa_prop -b True -ir msp_mined_seq_blastp_report.tsv -r bovine -s False -bsv msp_bs_viz.pdf
pred_dir = 'e:/safra/Documents/GitHub/optics/prediction_outputs/mnm_opsins/optics_on_all_dbs_2024-12-12_11-49-37'
optics_pred_file = f'{pred_dir}/all_dbs_predictions.tsv'
optics = pd.read_csv(optics_pred_file, sep='\t')

optics.head()

,Names,Single_Prediction,Prediction_Means,Prediction_Medians,Prediction_Lower_Bounds,Prediction_Upper_Bounds,Std_Deviation,%Identity_Nearest_VPOD_Sequence,Sequence_Length,Lmax_Hex_Color
0,MN519158.1,514.1,514.8,515.9,495.4,529.0,10.8,81.408,367,#1eff00
1,MN519147.1,490.0,499.7,500.0,491.0,506.6,4.0,89.266,367,#00ff96
2,XM_022258443.2,385.6,428.1,429.6,369.9,476.4,29.3,50.0,565,#3e00e2
3,AB208675.1,446.2,445.3,446.4,428.6,456.2,6.9,99.737,393,#002aff
4,AB208674.1,422.3,423.9,421.0,419.1,450.8,8.1,99.733,388,#4b00d8


In [5]:
optics.shape

(3359, 10)

In [4]:
source_file = './data_sources/lmax/cleaned_vpod_comp_accessory_dbs_2024-12-17_14-17-46.csv'
comp_db = pd.read_csv(source_file)
comp_db.head()

,comp_db_id,Full_Species,Accession,maxid,longcore_id,murwes_id,caves_id,porter2005_id,porter2006_id,LambdaMax
0,0,Drosophila melanogaster,Z86118,NaN,NaN,NaN,NaN,NaN,42.0,508.0
1,1,Loligo subulata,Z49108,NaN,NaN,NaN,NaN,NaN,2.0,499.0
2,2,Schistocerca gregaria,X80072,NaN,NaN,NaN,NaN,NaN,46.0,430.0
3,3,Schistocerca gregaria,X80071,NaN,NaN,NaN,NaN,NaN,39.0,520.0
4,4,Sphodromantis sp.,X71665,NaN,NaN,NaN,NaN,NaN,40.0,515.0


In [14]:
comp_db.iloc[21]

Full_Species     Neomysis americana
Accession         DQ852592–DQ852598
maxid                           NaN
longcore_id                     NaN
murwes_id                       NaN
caves_id                        NaN
porter2005_id                   NaN
porter2006_id                  18.0
LambdaMax                     520.0
Name: 21, dtype: object

In [5]:
final_err_filtered_df = mine_n_match(report_dir, source_file, ncbi_q_file, optics_pred_file, out='vpod_acc_dbs', err_filter = 15)

There were 0 unmatched species


In [6]:
final_err_filtered_df.head()

,Accession,Genus,Species,%Identity_Nearest_VPOD_Sequence,prediction_value,LambdaMax,abs_diff,comp_db_id,Protein,Gene_Description,Notes
mnm_id,,,,,,,,,,,
0,XM_034071738.1,Melopsittacus,undulatus,92.437,558.7,566.0,7.3,2683,MAAAWPAVMAARRRHEDEETTRDSVFTYTNSNNTRGPFDGPNYHIA...,PREDICTED: Melopsittacus undulatus red-sensiti...,NaN
1,KF134493.1,Platycercus,elegans,92.997,560.4,567.0,6.6,2617,MAAAWPAVMAARRRHEDEETTRDSVFTYTNSNNTRGPFDGPNYHIA...,"Platycercus elegans LWS opsin mRNA, complete cds",NaN
2,AB725207.1,Graphium,sarpedon,80.163,448.5,460.0,11.5,1441,MAANYTDDIGPMAYPMKLVSSEMVEHMMGWNIPEEHRDMVHEHWRN...,Graphium sarpedon GsB mRNA for B-sensitive ops...,NaN
3,NM_205440.2,Gallus,gallus,95.856,567.0,569.0,2.0,2497,MAAWEAAFAARRRHEEEDTTRDSVFTYTNSNNTRGPFEGPNYHIAP...,"Gallus gallus opsin 1 (cone pigments), long-wa...",NaN
4,JF823560.1,Poecilia,mexicana,89.607,530.6,537.0,6.4,1091,MAEDWGKQAFAPWKNNEETTRGFAFTYTNSNHTRDPFEGPNYHIAP...,Poecilia mexicana opsin locus R long-wave sens...,NaN


In [12]:
final_err_filtered_df.shape

(402, 11)

In [20]:
df_2

,comp_db_id,Full_Species,Accession,maxid,longcore_id,murwes_id,caves_id,porter2005_id,porter2006_id,LambdaMax
0,0,Drosophila melanogaster,Z86118,NaN,NaN,NaN,NaN,NaN,42.0,508.0
1,1,Loligo subulata,Z49108,NaN,NaN,NaN,NaN,NaN,2.0,499.0
2,2,Schistocerca gregaria,X80072,NaN,NaN,NaN,NaN,NaN,46.0,430.0
3,3,Schistocerca gregaria,X80071,NaN,NaN,NaN,NaN,NaN,39.0,520.0
4,4,Sphodromantis sp.,X71665,NaN,NaN,NaN,NaN,NaN,40.0,515.0
...,...,...,...,...,...,...,...,...,...,...
3463,3463,Heliconius erato petiverana,NaN,1563.0,NaN,NaN,NaN,NaN,NaN,390.0
3464,3464,Heliconius erato petiverana,NaN,1564.0,NaN,NaN,NaN,NaN,NaN,469.0
3465,3465,Heliconius erato petiverana,NaN,1565.0,NaN,NaN,NaN,NaN,NaN,556.0
3466,3466,Pieris rapae crucivora,NaN,1569.0,NaN,NaN,NaN,NaN,NaN,563.0


In [12]:
# Replace the missing values in the `Accession` column with an empty string
df_2 = comp_db.copy()
df_2= df_2[~df_2["Accession"].isna()].reset_index(drop = True)

# Filter the Accession column to include only those without '-' or '–'
df_2 = df_2[~df_2["Accession"].str.contains("-|–")].reset_index(drop = True)

# Create a list of `Accession` in `result_df`
accession_list = final_err_filtered_df["Accession"].to_list()

# Filter `df_2` to include only those whose `Accession` is not in `accession_list`
df_2_filtered = df_2[~df_2["Accession"].isin(accession_list)].reset_index(drop = True)


In [7]:
prot_list = get_prots_from_acc(df_2_filtered['Accession'].to_list())

In [13]:
df_2_filtered['Protein'] = prot_list

In [14]:
# Split the `Full_Species` column by the first space (' ') into `Genus` and `Species`
g_list = [x.split(' ')[0] for x in df_2_filtered['Full_Species']]
sp_list = [' '.join(x.split(' ')[1:]) for x in df_2_filtered['Full_Species']]
sp_list = [' '.join(x.split(' ')[0:]) for x in sp_list]

df_2_filtered["Genus"] = g_list
df_2_filtered["Species"] = sp_list

# Take the `comp_db_id`, `Genus`, `Species`, `Accession`, and `LambdaMax` from the filtered dataframe and rename the `LambdaMax` column to `closest_measurement`
df_2_filtered = df_2_filtered[["comp_db_id", "Genus", "Species", "Accession", "LambdaMax", "Protein"]]

# Add a column named `Notes` with the text 'Known sequence specified in accessory database'
df_2_filtered["Notes"] = "Known sequence specified in accessory database"

# Concatenate this series with the `result_df` dataframe
result_df2 = pd.concat([final_err_filtered_df, df_2_filtered]).reset_index(drop = True)
result_df2.index.name = 'mnm_id'


In [15]:
result_df2.head()

,Accession,Genus,Species,%Identity_Nearest_VPOD_Sequence,prediction_value,LambdaMax,abs_diff,comp_db_id,Protein,Gene_Description,Notes
mnm_id,,,,,,,,,,,
0,XM_034071738.1,Melopsittacus,undulatus,92.437,558.7,566.0,7.3,2683,MAAAWPAVMAARRRHEDEETTRDSVFTYTNSNNTRGPFDGPNYHIA...,PREDICTED: Melopsittacus undulatus red-sensiti...,NaN
1,KF134493.1,Platycercus,elegans,92.997,560.4,567.0,6.6,2617,MAAAWPAVMAARRRHEDEETTRDSVFTYTNSNNTRGPFDGPNYHIA...,"Platycercus elegans LWS opsin mRNA, complete cds",NaN
2,AB725207.1,Graphium,sarpedon,80.163,448.5,460.0,11.5,1441,MAANYTDDIGPMAYPMKLVSSEMVEHMMGWNIPEEHRDMVHEHWRN...,Graphium sarpedon GsB mRNA for B-sensitive ops...,NaN
3,NM_205440.2,Gallus,gallus,95.856,567.0,569.0,2.0,2497,MAAWEAAFAARRRHEEEDTTRDSVFTYTNSNNTRGPFEGPNYHIAP...,"Gallus gallus opsin 1 (cone pigments), long-wa...",NaN
4,JF823560.1,Poecilia,mexicana,89.607,530.6,537.0,6.4,1091,MAEDWGKQAFAPWKNNEETTRGFAFTYTNSNHTRDPFEGPNYHIAP...,Poecilia mexicana opsin locus R long-wave sens...,NaN


In [16]:
result_df2.to_csv('mnm_on_vpod_acc_dbs_final_results_fully_filtered_2.csv')

## Potential Solution for Species Synonyms

In [ ]:
#test code for getting species synonyms for ncbi search... 
#maybe add a spelling correction for species names too?
# Example usage
species_list = ["Homo sapiens", "Mus musculus", "Drosophila melanogaster"]
for species in species_list:
    synonyms = get_species_synonyms(species)
    print(f"{species} synonyms: {synonyms}")

    # Now use both the original name and synonyms in your NCBI query
    all_names = [species] + synonyms
    for name in all_names:
        # Perform your Entrez search for opsins here (e.g., using Entrez.esearch and Entrez.efetch)
        # ...